In [ ]:
#Sınfılandırma için gerekli kütüphanlerin ekleyelim

import torch
import torchvision
import torchvision.transforms as transforms # Görüntü transform etme için gerekli importlar
import torch.nn as nn
import torch.optim as optim
import os,fnmatch
import pandas as pd # cvs dosyalarını okumak için
from torch.utils.data import (Dataset,DataLoader)
from skimage import io
import time
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Colab Bağlantısı
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


*italik metin*#Veriyi Dahil Etme

In [ ]:
class veri(Dataset):
  def __init__(self,csv_file,rood_dir,transform=None):
    self.annotations= pd.read_csv(csv_file)
    self.root_dir = rood_dir
    self.transform = transform

  def __len__(self):
    return len(self.annotations)
  def __getitem__(self,index):
    img_path = os.path.join(self.root_dir,self.annotations.iloc[index,0])
    image= io.imread(img_path)
    y_label = torch.tensor(int(self.annotations.iloc[index,1]))

    if self.transform:
      image = self.transform(image)

      return (image,y_label)


# Veriyi Hazırlama

In [ ]:
dataset=veri(csv_file=r"/content/gdrive/MyDrive/Bistek/colab.csv",
         rood_dir=r"/content/gdrive/MyDrive/Bistek/karma",
         transform=torchvision.transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize(size=(28,28)),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

    ]))
# Compose en önemlisidir birden fazla dönüş(boyut,renk vs) işlemi içerir.
dataset

Veri Ön İşleme

In [ ]:
#tüm veriyi test ve train olarak veriyi bölüyoruz (0-60 aralığındadır veri)
train_set,test_set=torch.utils.data.random_split(dataset,[47,12])
#shuffle veriyi karıştırma işine yarar
train_loader=DataLoader(dataset=train_set,batch_size=1,shuffle=False)
test_loader=DataLoader(dataset=test_set,batch_size=1,shuffle=False)

Model Eğitimi

In [ ]:
class Net(nn.Module):
    def __init__(self):
          super(Net,self).__init__()
          self.conv1=nn.Conv2d(in_channels=3,out_channels=4,kernel_size=(5,5))
          self.conv2=nn.Conv2d(in_channels=4,out_channels=8,kernel_size=(3,3))
          self.conv3=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(2,2))
          self.conv4=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=(2,2))
          self.max=nn.MaxPool2d(kernel_size=(2,2))
          self.func=nn.ELU()#0 altına düşebilecek değerleride kapsaması için
          self.fc1=nn.Linear(in_features=32,out_features=50)#Gizli katmanımız
          self.fc2=nn.Linear(in_features=50,out_features=50)
          self.fc3=nn.Linear(in_features=50,out_features=100)
          self.fc4=nn.Linear(in_features=100,out_features=4)
    def forward(self,x):
          x=self.conv1(x)
          x=self.func(x)
          x=self.max(x)
          x=self.Conv2d(x)
          x=self.func(x)
          x=self.max(x)
          x=self.conv3(x)
          x=self.func(x)
          x=self.max(x)
          x=self.conv4(x)
          x=self.func(x)
          x=self.max(x)
          #flatten katmanını tanımlayalım
          x=x.view(x.size(0),-1)#düz bir hale getirmek için
          x=self.fc1(x)
          x=self.func(x)
          x=self.fc2(x)
          x=self.func(x)
          x=self.fc3(x)
          x=self.func(x)
          x=self.fc4(x)
          return x


In [ ]:
start=time.time()
model=Net()
optimizer=torch.optim.Adamax(model.parameters(),lr=0.001)
error=torch.nn.CrossEntropyLoss()
epoch=10
for i in range(epoch):
  for i,(images,label) in enumerate(train_loader):
        optimizer.zero_grad() # epoch değerleri kadar ilerleme yapılacak ve gradyanların üst üste binmesini önlemek için türevleri sıfırlama yapmamız gerekiyor
        tahmin=model(images)
        loss=error(tahmin,label)
        loss.backward()
        optimizer.step()
        print("Epoch [{}/{}],loss:{:4f}".format(epoch+1,epoch,loss.item()))#Êkok değerine göre bir oran vericek bize

end=time.time()
print("süre:",end-start)

Modelin Test Edilmesi

In [ ]:
start=time.time()
model=Net()
optimizer=torch.optim.Adamax(model.parameters(),lr=0.001)
error=torch.nn.CrossEntropyLoss()
epoch=10

kayıp=[]
count=10
iterasyon=[]
for i in range(epoch):
  for i,(images) in enumerate(train_loader):
    tahmin=model(images)
    optimizer.zero_grad()    # Eğitim kısmıdır
    loss=error(tahmin,label)
    loss.backward()
    optimizer.step()

    count+=1   # Modelimizde veriyi veriyoruz ve hata doğruluk göre hesaplıyor
    if count %100==0:
        total=0
        correct=0
        correct_hata=0

        for image,label in test_loader:
          out=model(images)
          pred=torch.max(out.data,1)[1]
          total+=len(label)

          correct+=(pred==label).sum()

          dogruluk=100*correct/float(total)

    if count %100==0:
      print("Iteration:{} Loss:{} Accuracy{}% Error: {}%".format(count,loss.data,dogruluk))






end=time.time()
print("süre:",end-start)